## 数据

Caption 的数据分为 instances.json 和 annotations.json.

instances.json 的 annotations 字段中有一个 images 列表，列表的每一个元素对应一张图片的信息，其中重要的是`id`和`image_id`:
+ id: 在 instances.json 中的 annotations 字段中 images 列表中每个条目的自身 id （ id 是这个条目的一个字段，整个条目是一个字典）
+ image_id: 列表中每个条目对应的图片的 ID

经过 cocoapi 返回的是一个字典 anns ， key:id, value:image 列表中该 id 对应的整个条目

那么(样本，类标 ) 或者(image, caption) 对的匹配方法就是通过从 anns 中取出来一个元素，得到 image_id

而对于 annotations.json 里面存储的是 image_id 和 caption 的对应关系。并且一个 image 对应多个 caption 。同样是调用 cocoapi （返回的东西名字相同，数据不一样），这时候侧重的就不同了。

对于 annotations.json 侧重的就是根据输入的 image_id 返回一个 anno_id 组成的列表。

---

实际上 instances.json 还包含图像 bbox （进行 detection 图像检测）、segmentation （进行图像分割），以及对应的图像名字。而 annotations.json 只是在 Caption 中需要的文件。

## 预处理

图片的预处理 RGB 这三个 Channel 的数值都进行标准化，然后映射到[0-1] 之间。

想把图片 resize 到 $256$，然后在剪裁、翻转之类的数据增强到指定的 $224$。

统计所有 caption 训练数据中单词的出现次数，如果一个 caption 中的任意一个单词的出现次数小于`vocab_threshold`，则从数据集中删除这个 caption 。
所以在训练的时候是根据 caption 一条条训练（根据 caption 找对应的 image) 。

## 训练

### data_loader.dataset

类似于 Pytorch 的 DataLoader ，传入数据集和`batch_size`之后返回的变量有一个`dataset`成员一样。

```python
train_dl = DataLoader(train_ds, batch_size=bs)
```

定义一个 get_loader 函数和 CoCoDataset 类。 get_loader 返回 Pytorch DataLoader 实例（参数是 COCODataset 实例，与 Pytorch 的 DataLoader 接口一致）。

COCODataset 有一个 vocab 成员，这个成员有两个 dict 方别存放 word->id, id->word 的映射关系，也就是每个参与训练的单词有自己的 id ，包括 start_word, end_word, unknown_word. 这个 vocab 可以保存下来，为下次训练时使用（vocab 只和 vocab_threshold 有关)，以备下次网格训练时使用。

COCODataset 还会生成一个列表（ caption_lengths ），每个元素是遍历 annotation 时该 caption 的单词的个数。然后训练抽样的方法是：从 caption_lengths 中选择一个值，然后列出所有样本中包含的 wrod 个数都为这个值的样本。在从这些样本中随即选择 batch_size 个进行训练。

之所以这样是因为如此才能对 NLP 进行批训练。不然每个样本的长度不一样，就只能一条一条的 caption 进行训练（因为不同长度 Pytorch 生成的动态图不一样，那么就无法在同一个批次进行训练）。如果那样无法享受 GPU 的加速。

> 另一中解决方法就是对进行短的样本进行填充（这样内存和训练时间都会增加）。Pytorch现在有对应的解决方法：PackedSequence（会记住每个样本的长度）。

COCODataset 实现了 \_\_getitem\_\_ 接口接收一个 annotation 索引，然后返回这个 annotation 对应的进过预处理的图片和经过分词得到的单词对应的 id 组成的 list.

然后就把图片送入到 EncoderCNN 中来抽取信息，然后通过 DecoderRNN 来生成 caption. 而训练就是通过计算生成的 caption 和 类标的损失来学习参数。

### EncoderCNN

图像特征的抽取网络使用的resnet50，去掉了最后的全连接层。抽取的特征维度为$(\text{batch_size}, 2048, 1, 1)$。然后将后面三个维度reshape为一个维度，即：$(\text{batch_size}, 2048)$。再通过全连接层实现：$2048 \rightarrow \text{embed_size}$。

### 近似搜索算法: Beam Search

束搜索相对于贪婪搜索的区别：

+ 贪婪搜索生成的最终的序列，在选择其中的每一个单词的时候，都是选择的预测的时候 softmax 最大的那个单词。
+ 对于参数 B=3 的束搜索，对于每个存在的生成序列，当预测下一个单词的时候，保留 B 个 softmax 最大的单词。也就是说每个序列预测了一个新的单词之后，这个序列会生成 B 个新的序列。那么如果之前就存在 B 个序列，则预测了一个单词之后会生成 B×B 个序列， 束搜索会保留新生成的序列中概率最高的 B 个序列去预测下一个单词。
  + 因为对于解码网络给定输入会一直存在 B 个预测序列，所以叫做束搜索。而参数 B 为 Beam Width，束宽。
  + 因为softmax值大多是很小的值，所以在对序列进行排名的时候，是取每个单词概率值取对数之后相加。
  + 这样的话因为softmax值的对数都是负值，那么算法更倾向于选择单词个数少的句子。一个做法是将序列概率的对数和比上句子的长度，更加通用的是给句子的长度添加一个 $\alpha \in [0, 1]$ 的指数，当 $\alpha$ 表示我们倾向于短句子的程度，当 $\alpha=1$ 的时候表示没有 bias，通常的取值为 $\alpha=0.7$。
  + B 的可取值：$1, 3, 10, 100, 1000, 3000$。
    + 当 B=1 的时候，就等价于贪婪搜索。

另外， Beam 搜索用于测试的时候。

### 参数意义

$totoal\_step = \lceil\frac{caption\_lengths}{batch\_size}\rceil$，$caption\_lengths$是训练数据中 annotations 的条数。

### attention

![attention](images/attention.png)

Attention 的存在就是在判断输出的下一个 word 的时候 CNN 输出的特征的重要性是不一样的。上图中的 features 的维度是 $(batch\_size, 14, 14, 2048)$，我们假设 hidden layer 的维度为$hidden\_dim$，attention layer（红色方框）的维度为$attention\_dim$。则图像 features 需要先将$(batch\_size, 14, 14, 2048) \rightarrow (batch\_size, 196, 2048)$，然后通过全连接层将$2048 \rightarrow attention\_dim$；同样需要将 hidden layer 通过全连接进行转换，即 $(batch\_size, hidden\_dim) \rightarrow (batch\_size, attention\_dim)$，并广播为$(batch\_size, 196, attention\_dim)$。

Soft Attention 就是将二者相加得到的 tensor 的维度仍然是$(batch\_size, 196, attention\_dim)$，然后通过一个全连接层降维至$(batch\_size, 196, 1)$。然后，`squeeze`为两维来计算每个像素的重要性（`softmax`）。再将其`unsqueeze`并广播为$(batch\_size, 196, 2048)$再和图像 feature 对应位置相乘，之后的结果在第二个维度上求和即：$\text{.sum(dim=1)}$此时的维度为 $batch\_size, 2048$ 即为 DecoderRNN 的输入。

Attention 解决的问题其实就是将 CNN 输出的三维向量更好的转换成二维的问题，即 $(batch\_size, 196, 2048) \rightarrow (batch\_size, ?)$。当然，可以直接将其展开为二维，一个就是输入维度太大。这样 LSTM 里面的全连接参数就比较多。再一个就失去了空间信息。通过 hidder layer （也就是细胞的状态）得到 features 每一个像素点对于下个 word 预测的重要性。也即得到每个像素点对应的 `dim=3` 的 $2048$ 个数值重要性（都对应同一个值）。然后 $2048$ 每个位子根据对应的 `softmax` 权重对 $196$ 个值求和求，得到最后的二维 `tensor`。

而此项目中 CNN 的输出是 $(batch\_size, 2048, 1, 1)$（输出大显存装不下。比如 resnet50 倒数第$3$层输出为$(batch\_size, 2048, 7, 7)$，然后接了一个 `ave pooling` 得到$(batch\_size, 2048, 1, 1)$，也是此项目最后使用的。而此项目$batch\_size=32$ `2G`显存刚好，如果不经过 `pooling` 那么这部分的内存占用会扩大$49$倍，很可能$batch\_size$只能是个位数。），直接就是二维的。

Attention 实现如下：

```python
class Attention(nn.Module):
    """
    Attention Network.
    """

    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        """
        :param encoder_dim: feature size of encoded images
        :param decoder_dim: size of decoder's RNN
        :param attention_dim: size of the attention network
        """
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # linear layer to transform encoded image
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # linear layer to transform decoder's output
        self.full_att = nn.Linear(attention_dim, 1)  # linear layer to calculate values to be softmax-ed
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  # softmax layer to calculate weights

    def forward(self, encoder_out, decoder_hidden):
        """
        Forward propagation.

        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, decoder_dim)
        :return: attention weighted encoding, weights
        """
        att1 = self.encoder_att(encoder_out)  # (batch_size, num_pixels, attention_dim)
        att2 = self.decoder_att(decoder_hidden)  # (batch_size, attention_dim)
        att = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)  # (batch_size, num_pixels)
        alpha = self.softmax(att)  # (batch_size, num_pixels)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, encoder_dim)

        return attention_weighted_encoding, alpha
```

### 损失函数

本项目使用的是`CrossEntropyLoss`损失函数。

1. `NLLLoss`: 负对数（似然）损失`(N)egetive (L)og (L)ikelihood (L)oss`。
2. `CrossEntropyLoss`: 交叉熵损失就是执行了 softmax 之后的 `NLLLoss`。

`NLLLoss(input, target)`只是根据类标`target`取`input`中对应的分类概率值再求和就可以了。也就是说在使用 `NLLLoss` 之前需要先将输入转换成概率然后再对每个概率取对数，而如果转换概率的方式为 `softmax` 的话就和对前一层（比如全连接层）的输出直接使用 `CrossEntropyLoss` 一样了。

也就是说`CrossEntropyLoss = NLLLoss(log_softmax(input), target)`。而如果使用的不是`softmax`则二者不同，比如`softmax`可以替换为`sigmoid`。

#### 例子

```python
input = torch.randn(3, 3)  # 每一行为对一个样本在3个类别上的预测值
softmax = nn.Softmax(dim=1)
target = torch.tensor([0, 2, 1])  # 每行对应样本的类标

loss = nn.CrossEntropyLoss()
temp = -torch.log(softmax(input))
# 下面的输出值相同
print(loss(input, target))
print((temp[0][0] + temp[1][2] + temp[2][1])/3)
```
#### 激活函数和代价函数的组合

+ 二次代价函数+线性神经元输出层
+ 交叉熵代价函数（两类的对数损失代价函数）+sigmoid神经元输出层
+ 对数损失代价函数+softmax输出层

而第二项中交叉熵代价函数其实就是两类时的负对数损失，而第三项就是 `CrossEntropyLoss`。也就是说负对数损失函数前面的激活函数既可以是 `sigmoid` 又可以是 `softmax`。


## 考点

### ResNet 为什么好？

残差结构保证了最少不会比浅层网络差。残差结构保证了不会梯度弥散，但是无法保证不会梯度爆炸（ReLU）。所以梯度弥散或爆炸还是被BN层解决的。

残差结构有点像差分放大器，使得残差结构可以对微小的变化敏感。图片像素点之间存在相关性，而梯度的相关性会随着网络深度的增加而降低。而经证明ResNet可以有效的减少这种相关性的衰减。

### 为什么设置 vocab_threshold？

偏差和方差的问题，不设置就容易过拟合。 vocab_threshold 设置为 $5$ 速度比设置为 $6$ 慢了 $2.5$ 倍。


## 数字图像处理关注的问题

<img src="images/image-process.jpg" alt="drawing" width="400" height="300"/>

1. 分类：这张图像中有气球。
2. 语义分割：这些像素全是气球像素。
3. 目标检测：这张图像中的这些位置上每个位置有一个气球（然后可以统计出一共有$7$个气球）。
4. 实例分割：这些位置上有$7$个（统计出来的）气球，并且这些像素分别属于每个气球。


## 图片预处理归一化的作用

1. 消除图像的共性，凸显个体差异
    其实也不只是图像，所有样本都是。比如下面的第一个图是样本在两个特征的分布：

    <img src="images/two-features.jpg" alt="drawing" width="400" height="250"/>

    而第二张图更加的鲜艳，更加突出了这张图中哪些位置存在边缘。

    <img src="images/scale-flower.jpg" alt="drawing" width="400" height="250"/>

2. 归一化与机器学习样本的区别
    图像的值在[0-255] ，方差基本一样。其实，只做去均值处理就可以了。

3. 梯度下降学习速度
    对于机器学习数据，会被量级大的特征 dominant ，训练过程容易震荡。
    而对于图像，当使用 relu 激活函数的时候，因为图像的输入值不做均值化都是正值，那么输出值也是正值，这样同一轮学习中所有参数的梯度根据损失函数也只能全是正或者全是负。而通常我们希望的是或者对于一个参数的学习路线是 Z 字形，可以理解为正样例学习一下，负样例学习一下（也可以是奇数轮学习和偶数轮学习）。

    <img src="images/zag-gradient.jpg" alt="drawing" width="400" height="250"/>

## refs

1. [MS-COCO 数据集的内容说明、数据的定义、标注信息 - 简书 ](https://www.jianshu.com/p/568a2f5195a9)
2. [答案解析(1)—史上最全 Transformer 面试题：灵魂 20 问帮你彻底搞定 Transformer - 知乎 ](https://zhuanlan.zhihu.com/p/149799951)
3. [深度学习中，图片的预处理为什么要减去图片的平均值，在什么情况下要选择这种预处理方式？ - blateyang 的回答 - 知乎 ](https://www.zhihu.com/question/49096923/answer/518032757)
4. [深度学习中，图片的预处理为什么要减去图片的平均值，在什么情况下要选择这种预处理方式？ - 徐之谓的回答 - 知乎 ](https://www.zhihu.com/question/49096923/answer/743038532)
5. [序列模型：Beam Search - 网易云课堂](https://mooc.study.163.com/learn/2001280005?tid=2001391038#/learn/content?type=detail&id=2001771063)


## 待解决

1. [LSTM细节分析理解（pytorch版） - 知乎](https://zhuanlan.zhihu.com/p/79064602)
2. [答案解析(1)—史上最全Transformer面试题：灵魂20问帮你彻底搞定Transformer - 知乎](https://zhuanlan.zhihu.com/p/149799951)
3. [ResNet 原理剖析 | 拾荒志](https://murphypei.github.io/blog/2020/06/why-resnet)